In [1]:
conda develop .

path exists, skipping /Users/paulinaschroder/Documents/IZA/Arbeit/gettsim
completed operation for: /Users/paulinaschroder/Documents/IZA/Arbeit/gettsim

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from itertools import product

from copy import deepcopy

from gettsim import set_up_policy_environment
from gettsim import compute_taxes_and_transfers

linestyles = itertools.cycle(["-",":","--","-."])

In [3]:
soep = pd.read_csv('/Users/paulinaschroder/Documents/Uni/Python/Bachelorarbeit/gettsim_data_soep.csv', sep=';')

/Users/paulinaschroder/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (94,95,99,110,112,148,156) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
soep.head()

,p_id,jahr,hh_id,index,zeiger_mutter,betreuungskost_m2,schulkosten,in_dataset_kidlong,hh_id_orig,hh_position,...,betreuungskost_m,entgeltpunkte,missing_hh_head,bruttolohn_m_hh,alter_hh_head,employment_status_hh_head,bruttolohn_m_sqr,alter_sqr,alter_hh_head_sqr,weiblich
0,901,2017,94.0,242741,NaN,0.0,0.0,0.0,94.0,Household head,...,0.0,0,False,0.0,66.0,Nicht erwerbstätig,0.0,4356.0,4356.0,1.0
1,1501,2017,159.0,242861,NaN,0.0,0.0,0.0,159.0,Household head,...,0.0,0,False,6500.0,59.0,Voll erwerbstätig,42250000.0,3481.0,3481.0,0.0
2,1601,2017,167.0,242897,NaN,0.0,0.0,0.0,167.0,Household head,...,0.0,0,False,700.0,77.0,Nicht erwerbstätig,0.0,5929.0,5929.0,0.0
3,1602,2017,167.0,242911,NaN,0.0,0.0,0.0,167.0,Spouse,...,0.0,0,False,700.0,77.0,Nicht erwerbstätig,490000.0,3136.0,5929.0,1.0
4,2301,2017,230.0,243142,NaN,0.0,0.0,0.0,230.0,Household head,...,0.0,0,False,0.0,71.0,Nicht erwerbstätig,0.0,5041.0,5041.0,0.0


In [5]:
soep.set_index(["hh_id", "p_id",'jahr']).sort_index()


index  zeiger_mutter  betreuungskost_m2  \
hh_id     p_id     jahr                                              
94.0      901      2017   242741            NaN                0.0   
                   2018   242742            NaN                0.0   
                   2019   242743            NaN                0.0   
159.0     1501     2017   242861            NaN                0.0   
                   2018   242862            NaN                0.0   
...                          ...            ...                ...   
3923878.0 39238705 2017   242496            NaN                NaN   
          39238706 2017   242497            NaN                NaN   
3923908.0 39005602 2019  1145019            NaN                0.0   
3923916.0 39107902 2019  1146553            NaN                0.0   
3923967.0 37362603 2019  1123501            NaN                0.0   

                         schulkosten  in_dataset_kidlong  hh_id_orig  \
hh_id     p_id     jahr                                                
94.0      901      2017          0.0                 0.0        94.0   
                   2018          0.0                 0.0        94.0   
                   2019          0.0                 0.0        94.0   
159.0     1501     2017          0.0                 0.0       159.0   
                   2018          0.0                 0.0       159.0   
...                              ...                 ...         ...   
3923878.0 39238705 2017          NaN                 1.0   3923878.0   
          39238706 2017          NaN                 1.0   3923878.0   
3923908.0 39005602 2019          0.0                 0.0   3900568.0   
3923916.0 39107902 2019          0.0                 0.0   3910792.0   
3923967.0 37362603 2019          0.0                 0.0   3736261.0   

                            hh_position                 hh_position_raw  \
hh_id     p_id     jahr                                                   
94.0      901      2017  Household head  Haushaltsvorstand,Bezugsperson   
                   2018  Household head  Haushaltsvorstand,Bezugsperson   
                   2019  Household head  Haushaltsvorstand,Bezugsperson   
159.0     1501     2017  Household head  Haushaltsvorstand,Bezugsperson   
                   2018  Household head  Haushaltsvorstand,Bezugsperson   
...                                 ...                             ...   
3923878.0 39238705 2017  Other relative    Leibliche/r Bruder,Schwester   
          39238706 2017  Other relative    Leibliche/r Bruder,Schwester   
3923908.0 39005602 2019  Household head  Haushaltsvorstand,Bezugsperson   
3923916.0 39107902 2019  Household head  Haushaltsvorstand,Bezugsperson   
3923967.0 37362603 2019  Household head  Haushaltsvorstand,Bezugsperson   

                         in_dataset_pbrutto geschlecht  ...  betreuungskost_m  \
hh_id     p_id     jahr                                 ...                     
94.0      901      2017                 1.0   weiblich  ...          0.000000   
                   2018                 1.0   weiblich  ...          0.000000   
                   2019                 1.0   weiblich  ...          0.000000   
159.0     1501     2017                 1.0   männlich  ...          0.000000   
                   2018                 1.0   männlich  ...          0.000000   
...                                     ...        ...  ...               ...   
3923878.0 39238705 2017                 1.0   weiblich  ...          0.025343   
          39238706 2017                 1.0   männlich  ...          0.075362   
3923908.0 39005602 2019                 1.0   weiblich  ...          0.000000   
3923916.0 39107902 2019                 1.0   männlich  ...          0.000000   
3923967.0 37362603 2019                 1.0   männlich  ...          0.000000   

                         entgeltpunkte  missing_hh_head  bruttolohn_m_hh  \
hh_id     p_id     jahr                                                  

In [6]:
np.count_nonzero(soep['soc_assist_eld'])

10590

In [7]:
soep1 = soep.loc[(soep['soc_assist_eld']!=0)]
soep1[["hh_id", "p_id",'jahr','soc_assist_eld']].set_index(["hh_id", "p_id",'jahr']).sort_index()

soc_assist_eld
hh_id     p_id     jahr                
604.0     6002     2017          1052.0
7641.0    76401    2018           750.0
13676.0   136701   2017          9600.0
                   2018         12000.0
                   2019         12000.0
...                                 ...
3923800.0 39238005 2017             NaN
3923835.0 39238301 2017             NaN
3923878.0 39238701 2017             NaN
          39238705 2017             NaN
          39238706 2017             NaN

[10590 rows x 1 columns]

In [8]:
params_dict, policy_func_dict = set_up_policy_environment("2016")

In [9]:
soep = soep.dropna(subset=['soc_assist_eld'])

In [10]:
soep = soep.fillna(0)

In [11]:
soep2 = soep.drop([
        "anz_kinder_hh",
        "anz_kinder_tu",
        "anz_minderj_hh",
        "nettolohn_m",
        ], axis=1)
soep2["hh_id"] = soep2["hh_id"].astype(int)
soep2["alter"] = soep2["alter"].astype(int)
soep2["arbeitsl_vorj_m"] = soep2["arbeitsl_vorj_m"].astype(int)
soep2["m_elterngeld"] = soep2["m_elterngeld"].astype(int)
soep2["behinderungsgrad"] = soep2["behinderungsgrad"].astype(int)
soep2["geburtsjahr"] = soep2["geburtsjahr"].astype(int)
soep2["geburtsmonat"] = soep2["geburtsmonat"].astype(int)
soep2["tu_id"] = soep2["tu_id"].astype(int)
soep2["kind"] = soep2["kind"].astype(bool)
soep2["arbeitsl_lfdj_m"] = soep2["arbeitsl_lfdj_m"].astype(int)
soep2["arbeitsl_vor2j_m"] = soep2["arbeitsl_vor2j_m"].astype(int)
soep2["immobilie_baujahr_hh"] = soep2["immobilie_baujahr_hh"].astype(int)
soep2["jahr_renteneintr"] = soep2["jahr_renteneintr"].astype(int)
soep2["m_elterngeld_mut"] = soep2["m_elterngeld_mut"].astype(int)
soep2["m_elterngeld_vat"] = soep2["m_elterngeld_vat"].astype(int)
soep2["wohnfläche_hh"] = soep2["wohnflaeche_hh"].astype(int)
soep2["arbeitsstunden_w"] = soep2["weekly_working_hours_actual"].astype(int)
soep2["prv_rente_m"] = soep2["prv_rente_beitr_m"]
soep2["grundrentenzeiten"] = 1
soep2["entgeltpunkte_grundrente"] = 1
soep2["gr_bewertungszeiten"] = 1
soep2["staatl_rente_m"] = soep2["ges_rente_m"]
#soep2["sonstig_eink_m"] = 0

In [12]:
soep3 = soep2.loc[(soep2['rentner']==True)]
soep_rentner = soep3.loc[(soep3["rented_or_owned"]!="Heimbewohner oder Gemeinschaftsunterkunft")]

In [13]:
soep17=soep_rentner.loc[(soep_rentner['jahr']==2017)].copy()
soep17["soc_assist_eld_m"] = soep17['soc_assist_eld']/12
soep17["sonstig_eink_m_diff"] = soep17["sonstig_eink_m"] - soep17["soc_assist_eld_m"]
soep17["sonstig_eink_m"] = soep17["sonstig_eink_m_diff"].round(2)

In [14]:
ergebnisse_personen = compute_taxes_and_transfers(
    data=soep17,
    functions=policy_func_dict,
    params=params_dict,
    targets=[
        "haushaltsgröße_hh",
        #"ges_rente_m_tu",
        "regelsatz_m_hh_ab_2011",
        "sozialv_beitr_m",
        "kost_unterk_m_hh",
        "grunds_im_alter_m_hh",
        "anrechenbares_eink_grunds_ia_m_hh",
        'anrechenbares_erwerbs_eink_grunds_ia_m',
        #'staatl_rente_m',
        'anrechenbare_prv_rente_grunds_ia_m',
        'kapital_eink_minus_pauschbetr',
        'elterngeld_m',
        "regelbedarf_m_grunds_ia_vermögens_check_hh",
        #"soli_st_tu",
        #"arbeitsl_geld_m",
        #"arbeitsl_geld_2_m_hh",
        #"kindergeld_m_tu",
        #"kindergeld_m_hh",
        #"wohngeld_m_hh",
        #"kinderzuschlag_m_hh",
        #"eink_st_tu",
    ],
    columns_overriding_functions=["staatl_rente_m"],
)
ergebnisse_personen = ergebnisse_personen.join(soep17[["hh_id","soc_assist_eld_m","alter","ges_rente_m","bruttolohn_m_hh","behinderungsgrad","hh_size","sonstig_eink_m","vermiet_eink_m","staatl_rente_m"]])

In [15]:
# Zahlen auf Haushaltsebene aggregieren, alles in Monate umrechnen.
ergebnisse = ergebnisse_personen.groupby('hh_id').agg(
    {
        'haushaltsgröße_hh': lambda x: x.sum(),
        'hh_size': lambda x: x.sum(),
        #'ges_rente_m_tu': lambda x: x.sum(),
        'ges_rente_m': lambda x: x.sum(),
        'staatl_rente_m': lambda x: x.sum(),
        'regelsatz_m_hh_ab_2011': lambda x: x.sum(),
        'kost_unterk_m_hh': lambda x: x.sum(),
        "anrechenbares_eink_grunds_ia_m_hh": lambda x: x.sum(),
        "regelbedarf_m_grunds_ia_vermögens_check_hh": lambda x: x.sum(),
        'anrechenbares_erwerbs_eink_grunds_ia_m': lambda x: x.sum(),
        'anrechenbare_prv_rente_grunds_ia_m': lambda x: x.sum(),
        'sonstig_eink_m': lambda x: x.sum(),
        'vermiet_eink_m': lambda x: x.sum(),
        #'kapital_eink_minus_pauschbetr': lambda x: x.sum() / 12,
        #'elterngeld_m': lambda x: x.sum(),
        #'bruttolohn_m_hh': lambda x: x.sum(),
        #'eink_st_tu': lambda x: x.max() / 12,
        #'soli_st_tu': lambda x: x.max() / 12,
        "sozialv_beitr_m": lambda x: x.sum(),
        #'kindergeld_m_hh': lambda x: x.max(),
        'alter': lambda x: x.max(),
        'behinderungsgrad': lambda x: x.sum(),
        'soc_assist_eld_m': lambda x: x.sum(),
        'grunds_im_alter_m_hh':lambda x: x.sum(),
        #'arbeitsl_geld_m': lambda x: x.sum(),
        #'arbeitsl_geld_2_m_hh': lambda x: x.max(),
        #'wohngeld_m_hh': lambda x: x.max(),
        #'kinderzuschlag_m_hh': lambda x: x.max(),
    }
).rename(
    columns={
        #'soc_assist_eld': 'soc_assist_eld_m',
    }
)

In [16]:
ergebnisse

,haushaltsgröße_hh,hh_size,ges_rente_m,staatl_rente_m,regelsatz_m_hh_ab_2011,kost_unterk_m_hh,anrechenbares_eink_grunds_ia_m_hh,regelbedarf_m_grunds_ia_vermögens_check_hh,anrechenbares_erwerbs_eink_grunds_ia_m,anrechenbare_prv_rente_grunds_ia_m,sonstig_eink_m,vermiet_eink_m,sozialv_beitr_m,alter,behinderungsgrad,soc_assist_eld_m,grunds_im_alter_m_hh
hh_id,,,,,,,,,,,,,,,,,
167,1,2.0,1574.0,1574.0,404.0,433.928571,1400.860000,0.000000,0.0,0.0,0.00,0.0,173.14000,77,70,0.000000,0.000000
230,1,1.0,3900.0,3900.0,404.0,0.000000,3448.581192,0.000000,0.0,0.0,0.00,50.0,419.25000,71,0,0.000000,0.000000
582,1,2.0,1850.0,1850.0,404.0,450.000000,7081.875000,0.000000,0.0,5698.0,0.00,0.0,466.12500,83,0,0.000000,0.000000
604,1,1.0,792.0,792.0,404.0,450.000000,706.860000,854.000000,0.0,0.0,0.00,0.0,85.14000,67,0,87.666667,147.140000
990,1,1.0,5300.0,5300.0,404.0,450.000000,4692.700889,0.000000,0.0,0.0,0.00,0.0,466.12500,73,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471240,4,4.0,2274.0,2274.0,1456.0,0.000000,4059.090000,0.000000,0.0,0.0,0.00,0.0,244.45500,76,0,0.000000,0.000000
3490091,1,5.0,1350.0,1350.0,404.0,0.000000,1204.875000,0.000000,0.0,0.0,0.00,0.0,145.12500,77,0,0.000000,0.000000
3500038,1,2.0,620.0,620.0,404.0,317.368421,604.600000,721.368421,0.0,0.0,51.25,0.0,66.65000,77,100,0.000000,116.768421


In [17]:
ergebnisse_1 = ergebnisse.loc[(ergebnisse['hh_size']==1)].copy()

In [18]:
ergebnisse_1["Grundsicherung_sim"] = (
    ergebnisse_1["regelsatz_m_hh_ab_2011"] 
    + ergebnisse_1["kost_unterk_m_hh"]
    + ergebnisse_1["sozialv_beitr_m"]
    - ergebnisse_1["ges_rente_m"])

ergebnisse_1.loc[ergebnisse_1['behinderungsgrad'] >= 50, 'Grundsicherung_sim'] = (
    ergebnisse_1["Grundsicherung_sim"] 
    + 0.17 * ergebnisse_1["regelsatz_m_hh_ab_2011"]
)

ergebnisse_1.loc[ergebnisse_1['Grundsicherung_sim'] <= 0, 'Grundsicherung_sim'] = 0

In [19]:
ergebnisse_1

,haushaltsgröße_hh,hh_size,ges_rente_m,staatl_rente_m,regelsatz_m_hh_ab_2011,kost_unterk_m_hh,anrechenbares_eink_grunds_ia_m_hh,regelbedarf_m_grunds_ia_vermögens_check_hh,anrechenbares_erwerbs_eink_grunds_ia_m,anrechenbare_prv_rente_grunds_ia_m,sonstig_eink_m,vermiet_eink_m,sozialv_beitr_m,alter,behinderungsgrad,soc_assist_eld_m,grunds_im_alter_m_hh,Grundsicherung_sim
hh_id,,,,,,,,,,,,,,,,,,
230,1,1.0,3900.000000,3900.000000,404.0,0.000000,3448.581192,0.00,0.0,0.0,0.0,50.0,419.250000,71,0,0.000000,0.0000,0.0000
604,1,1.0,792.000000,792.000000,404.0,450.000000,706.860000,854.00,0.0,0.0,0.0,0.0,85.140000,67,0,87.666667,147.1400,147.1400
990,1,1.0,5300.000000,5300.000000,404.0,450.000000,4692.700889,0.00,0.0,0.0,0.0,0.0,466.125000,73,0,0.000000,0.0000,0.0000
1015,1,1.0,2501.000000,2501.000000,404.0,421.875000,6748.875000,0.00,0.0,4714.0,0.0,0.0,466.125000,79,50,0.000000,0.0000,0.0000
1708,1,1.0,1400.000000,1400.000000,404.0,301.800000,1246.000000,0.00,0.0,0.0,0.0,0.0,154.000000,69,75,0.000000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3344850,1,1.0,400.000000,400.000000,404.0,450.000000,357.000000,854.00,0.0,0.0,0.0,0.0,43.000000,76,0,370.250000,497.0000,497.0000
3346667,1,1.0,1751.111111,1751.111111,404.0,420.000000,1562.866667,824.00,0.0,0.0,0.0,0.0,188.244444,67,0,0.000000,0.0000,0.0000
3414603,1,1.0,1709.000000,1709.000000,404.0,0.000000,18408.023810,0.00,0.0,19474.0,0.0,0.0,455.531250,69,0,0.000000,0.0000,0.0000


In [20]:
a = ergebnisse_1.loc[(ergebnisse_1['soc_assist_eld_m']!=0)]
a

,haushaltsgröße_hh,hh_size,ges_rente_m,staatl_rente_m,regelsatz_m_hh_ab_2011,kost_unterk_m_hh,anrechenbares_eink_grunds_ia_m_hh,regelbedarf_m_grunds_ia_vermögens_check_hh,anrechenbares_erwerbs_eink_grunds_ia_m,anrechenbare_prv_rente_grunds_ia_m,sonstig_eink_m,vermiet_eink_m,sozialv_beitr_m,alter,behinderungsgrad,soc_assist_eld_m,grunds_im_alter_m_hh,Grundsicherung_sim
hh_id,,,,,,,,,,,,,,,,,,
604,1,1.0,792.000000,792.000000,404.0,450.000000,706.860000,854.000000,0.0,0.0,0.00,0.0,85.140000,67,0,87.666667,147.140000,147.140000
37940,1,1.0,360.000000,360.000000,404.0,231.617647,321.300000,635.617647,0.0,0.0,0.00,0.0,38.700000,73,0,300.000000,314.317647,314.317647
71099,1,1.0,908.000000,908.000000,404.0,450.000000,1847.140000,854.000000,0.0,1198.0,0.00,0.0,258.860000,66,0,684.250000,0.000000,204.860000
91642,1,1.0,245.000000,245.000000,404.0,257.142857,218.050000,661.142857,0.0,0.0,0.00,0.0,26.950000,81,0,250.000000,443.092857,443.092857
128228,1,1.0,290.000000,290.000000,404.0,420.000000,258.825000,824.000000,0.0,0.0,0.00,0.0,31.175000,73,0,550.000000,565.175000,565.175000
208558,1,1.0,838.000000,838.000000,404.0,370.000000,949.150000,774.000000,0.0,0.0,203.33,0.0,92.180000,67,50,79.000000,0.000000,96.860000
254436,1,1.0,450.000000,450.000000,404.0,356.538462,401.625000,0.000000,0.0,0.0,0.00,0.0,48.375000,66,0,71.666667,0.000000,358.913462
257443,1,1.0,119.000000,119.000000,404.0,441.346154,106.207500,845.346154,0.0,0.0,0.00,0.0,12.792500,80,30,675.333333,739.138654,739.138654
259101,1,1.0,1296.083333,1296.083333,404.0,424.800000,2074.504375,828.800000,0.0,0.0,917.75,0.0,139.328958,65,80,305.583333,0.000000,0.000000


In [21]:
np.count_nonzero(a['hh_size'])

50

In [22]:
# in sonstig_eink_m ist soc_assist_eld